In [1]:
# 1. Install the required packages
# On Windows, you just need to execute this cell for once.
try:
    import google.colab
    # IN_COLAB = True
    %pip install --upgrade torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu126
except ImportError:
    # IN_COLAB = False

%pip install -q git+https://github.com/huggingface/transformers
%pip install -q git+https://github.com/huggingface/accelerate
%pip install -q git+https://github.com/huggingface/diffusers
%pip install -q gradio ftfy tensorboard
%pip install -q bitsandbytes
%pip install -q xformers --index-url https://download.pytorch.org/whl/cu126

# Install ipyfilechooser (if not already installed)
%pip -q install ipyfilechooser
print("Package installation finished.")

Looking in indexes: https://download.pytorch.org/whl/cu126
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.6.0+cu124 which is incompatible.
torchvision 0.23.0+cu126 requires torch==2.8.0, but you have torch 2.6.0+cu124 which is incompatible.
Package installation finished.


In [2]:
# 2. Create folders and download training scripts
import os, shutil
from pprint import pprint

model_dir = "./models"
embedding_dir = "./embeddings"
output_dir = "./output"

# Global variables
models = [
   "stable-diffusion-v1-5/stable-diffusion-v1-5",
   "Lykon/DreamShaper",
   "digiplay/DarkSushi2.5D_v1",
   "Dreamlike photoreal",
]

links = [
   "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.safetensors",
   "https://huggingface.co/Lykon/DreamShaper/blob/main/DreamShaper_6.2_BakedVae_pruned.safetensors",
   "https://huggingface.co/digiplay/DarkSushi2.5D_v1/blob/main/darkSushi25D25D_v10.safetensors",
   "https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/blob/main/dreamlike-photoreal-2.0.safetensors",
]

embeddings = []

# browse the model_dir directory and add found items to models and links
# browse the embedding_dir and add found items to embeddings

pipeline = None

# Create the directories if they don't exist
os.makedirs(model_dir, exist_ok=True)
os.makedirs(embedding_dir, exist_ok=True)
# Delete the 'output' folder and its contents
shutil.rmtree(output_dir, ignore_errors=True)
os.makedirs(output_dir, exist_ok=True)

def download_file(filename, url):
  # fetch train_dreambooth.py if it doesn't exist
  if not os.path.exists(filename):
      !wget "{url}"
  else:
      print(f"{filename} already exists, skipping download.")

download_file("lpw_stable_diffusion.py", "https://raw.githubusercontent.com/jomo0825/MrFuGenerativeAI/main/DiffusionModel/lpw_stable_diffusion.py")

def update_descriptors():
   # Browse the model_dir directory and add found items to models and links
   for filename in os.listdir(model_dir):
      if filename.endswith('.safetensors'):
         model_name = os.path.splitext(filename)[0]
         if model_name not in models:
            models.append(model_name)
         file_path = os.path.join(model_dir, filename)
         if file_path not in links:
            links.append(file_path)

   # Browse the embedding_dir and add found items to embeddings
   for filename in os.listdir(embedding_dir):
      if filename.endswith('.safetensors'):  # Assuming embeddings are .pt files
         embedding_name = os.path.splitext(filename)[0]
         if embedding_name not in embeddings:
            embeddings.append(embedding_name)

   # (Optional) Print the updated lists to verify
   pprint(models)
   pprint(links)
   pprint(embeddings)

update_descriptors()

--2025-09-11 13:19:57--  https://raw.githubusercontent.com/jomo0825/MrFuGenerativeAI/main/DiffusionModel/lpw_stable_diffusion.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74824 (73K) [text/plain]
Saving to: ‘lpw_stable_diffusion.py’

lpw_stable_diffusio 100%[===================>]  73.07K  --.-KB/s    in 0.01s   

2025-09-11 13:19:57 (5.59 MB/s) - ‘lpw_stable_diffusion.py’ saved [74824/74824]

['stable-diffusion-v1-5/stable-diffusion-v1-5',
 'Lykon/DreamShaper',
 'digiplay/DarkSushi2.5D_v1',
 'Dreamlike photoreal']
['https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.safetensors',
 'https://huggingface.co/Lykon/DreamShaper/blob/main/DreamShaper_6.2_BakedVae_pruned.safetensors',
 'https://huggingface.co/digip

In [4]:
# 3. Upload Dreambooth and TI models
import os
import shutil
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
import ipywidgets as widgets

# Define a function to move the selected file
def move_file(target_directory, file_path):
  if file_path is not None:
      filename = os.path.basename(file_path)
      destination_path = os.path.join(target_directory, filename)
      # test if the destination_path alread exist
      if not os.path.exists(destination_path):
        shutil.copy(file_path, destination_path)
        print(f"Copy {filename} to {target_directory}")
      else:
        print(f"{filename} already exists.")
  else:
      print("No file selected.")

def upload_model(b):
    with output:
        output.clear_output()
        move_file(model_dir, chooserDB.selected)
        # add_DB(os.path.basename(chooserDB.selected))
        update_descriptors()

def upload_TI(b):
    with output:
        output.clear_output()
        move_file(embedding_dir, chooserTI.selected)
        # add_TI(os.path.basename(chooserTI.selected))
        update_descriptors()

try:
    from google.colab import drive
    IN_COLAB = True
    # Mount Google Drive
    drive.mount('/content/drive')
    # Define source and target directories
    source_directory_DB = '/content/drive/MyDrive/Dreambooth'
    source_directory_TI = '/content/drive/MyDrive/TextualInversion'
except ImportError:
    IN_COLAB = False
    source_directory_DB = "./"
    source_directory_TI = "./"

target_directory = model_dir  # Ensure model_dir is defined

# Create and display the file chooser widget
chooserDB = FileChooser(source_directory_DB)
chooserDB.title = '<b>Select a <span style="color:red">Dreambooth</span> model to upload:</b>'
display(chooserDB)
# Add a button to trigger the file transfer
DB_button = widgets.Button(description="Upload Model")
DB_button.on_click(upload_model)
display(DB_button)

# Create and display the file chooser widget
chooserTI = FileChooser(source_directory_TI)
chooserTI.title = '<b>Select a <span style="color:blue">Textual Inversion</span> embedding to upload:</b>'
display(chooserTI)
TI_button = widgets.Button(description="Upload Embedding")
TI_button.on_click(upload_TI)
display(TI_button)
output = widgets.Output()
display(output)

Mounted at /content/drive


InvalidPathError: /content/drive/MyDrive/Dreambooth does not exist

In [5]:
# 4. Open WebUI
import torch
import random
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler,EulerAncestralDiscreteScheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
import gradio as gr
from PIL import Image
from lpw_stable_diffusion import StableDiffusionLongPromptWeightingPipeline

def load_pipeline(model_name, progress=gr.Progress()):
    progress(0, desc="Loading text2img pipeline...")
    # In this project we only use .safetensors model file. No pretrained model folders.
    # pipe_txt2img = StableDiffusionPipeline.from_single_file(
    pipe_txt2img = StableDiffusionLongPromptWeightingPipeline.from_single_file(
        model_name,
        torch_dtype=torch.float16,
        safety_checker=StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
        use_safetensors=True
    ).to("cuda")

    pipe_txt2img.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_txt2img.scheduler.config)

    progress(0.5, desc="Loading embeddings...")
    for embedding in embeddings:
        path = os.path.join(embedding_dir, f"{embedding}.safetensors")
        path = os.path.normpath(path)
        print(path)
        pipe_txt2img.load_textual_inversion(path)


    # pipe_img2img = StableDiffusionImg2ImgPipeline(
    #     vae=pipe_txt2img.vae,
    #     text_encoder=pipe_txt2img.text_encoder,
    #     tokenizer=pipe_txt2img.tokenizer,
    #     unet=pipe_txt2img.unet,
    #     scheduler=pipe_txt2img.scheduler,
    #     safety_checker=pipe_txt2img.safety_checker,  # This will get the safety checker from txt2img
    #     feature_extractor=pipe_txt2img.feature_extractor,  # This will get the feature extractor from txt2img
    # ).to("cuda")
    # pipe_img2img.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_img2img.scheduler.config)
    progress(1.0, desc="Done!")
    return pipe_txt2img

def txt2img(prompt, negative_prompt, height, width, steps, guidance_scale, seed):
    global pipeline
    if seed == -1:
        seed = random.randint(0, 2**32 - 1)
    generator = torch.Generator(device="cuda").manual_seed(seed)
    output = pipeline(
        prompt,
        negative_prompt=negative_prompt,
        height=height,
        width=width,
        num_inference_steps=steps,
        guidance_scale=guidance_scale,
        generator=generator
    )

    if not output.nsfw_content_detected[0]:
      #print("Not NSFW")
      return output.images[0], "Success", f"Noise seed: {seed}"
    else:
      #print("NSFW Detected")
      return output.images[0], "NSFW content detected", "NSFW content detected"

    #if len(output.images) == 0:
    #   return None, "NSFW content detected"
    #return output.images[0], "Success"

def img2img(init_image, prompt, negative_prompt, height, width, strength, steps, guidance_scale, seed):
    global pipeline
    if init_image is None:
        return None
    if seed == -1:
        seed = random.randint(0, 2**32 - 1)
    generator = torch.Generator(device="cuda").manual_seed(seed)
    init_image = Image.fromarray(init_image)
    init_image = init_image.resize((width, height), Image.Resampling.NEAREST)

    output = pipeline.img2img(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance_scale,
        generator=generator
    )

    if not output.nsfw_content_detected[0]:
      #print("Not NSFW")
      return output.images[0], "Success"
    else:
      #print("NSFW Detected")
      return output.images[0], "NSFW content detected"

def ui():
    with gr.Blocks(css="""
    .absolute-top-right {
    position: absolute !important;
    top: 27px;
    right: 2px;
    z-index: 100;
    }
    .absolute-top-right button {
    min-width: 32px !important;
    height: 32px !important;
    padding: 0 !important;
    position: absolute;
    left: 4px;
    }
    """) as demo:
        global pipeline
        with gr.Row():
            model_dropdown = gr.Dropdown(choices=models, value=models[0], label="Model", interactive=True)
            loading_status = gr.Textbox(value="Ready", label="Status", interactive=False)

        # load the initial model
        pipeline = load_pipeline(links[0])

        def on_model_change(model_name):
            global pipeline
            index = models.index(model_name)
            pipeline = load_pipeline(links[index])
            print(f"Model changed to: {model_name}")
            return "Ready"

        model_dropdown.change(
        fn=on_model_change,
        inputs=[model_dropdown],
        outputs=[loading_status],
        queue=False
        )

        # with gr.Row():
        #   txtCustomModel = gr.Textbox(label="Custom Model", placeholder="")
        with gr.Tabs():
            with gr.Tab("Text to Image"):
                with gr.Row():
                    with gr.Column():
                        txt2img_prompt = gr.Textbox(label="Prompt", value="a girl in beautiful green valley and blue skies")
                        txt2img_neg_prompt = gr.Textbox(label="Negative Prompt", value="nsfw, ugly, noise, blank, blurry, mutation, mangled")
                        txt2img_width = gr.Slider(128, 1024, value=512, step=64, label="Width")
                        txt2img_height = gr.Slider(128, 1024, value=512, step=64, label="Height")
                        txt2img_steps = gr.Slider(1, 100, value=20, step=1, label="Steps")
                        txt2img_guidance = gr.Slider(1, 20, value=7.0, step=0.1, label="Guidance Scale")
                        txt2img_seed = gr.Number(value=-1, label="Seed")

                    with gr.Column():
                        txt2img_generate = gr.Button("Generate", variant="primary")
                        txt2img_output = gr.Image(label="Output")
                        generation_status = gr.TextArea(label="Generation Status")

                txt2img_generate.click(
                    fn=txt2img,
                    inputs=[
                        txt2img_prompt,
                        txt2img_neg_prompt,
                        txt2img_height,
                        txt2img_width,
                        txt2img_steps,
                        txt2img_guidance,
                        txt2img_seed
                    ],
                    outputs=[txt2img_output,loading_status, generation_status]
                )
                txt2img_generate.interactive = loading_status.value == "Ready"


            with gr.Tab("Image to Image"):
                with gr.Row():
                    with gr.Column():
                        img2img_input = gr.Image(label="Input Image", type="numpy")
                        img2img_prompt = gr.Textbox(label="Prompt", value="a girl in beautiful green valley and blue skies")
                        img2img_neg_prompt = gr.Textbox(label="Negative Prompt", value="nsfw, ugly, noise, blank, blurry, mutation, mangled")
                        img2img_height = gr.Slider(128, 2048, value=512, step=64, label="Height")
                        img2img_width = gr.Slider(128, 2048, value=512, step=64, label="Width")
                        img2img_strength = gr.Slider(0, 1, value=0.75, step=0.01, label="Strength")
                        img2img_steps = gr.Slider(1, 100, value=20, step=1, label="Steps")
                        img2img_guidance = gr.Slider(1, 20, value=7.0, step=0.1, label="Guidance Scale")
                        img2img_seed = gr.Number(value=-1, label="Seed")

                    with gr.Column():
                        img2img_generate = gr.Button("Generate", variant="primary")
                        img2img_output = gr.Image(label="Output")
                        with gr.Row(elem_classes="absolute-top-right"):  # Position button
                            load_to_input = gr.Button("⬅️", scale=0)  # Small arrow button

                img2img_generate.click(
                    fn=img2img,
                    inputs=[
                        img2img_input,
                        img2img_prompt,
                        img2img_neg_prompt,
                        img2img_height,
                        img2img_width,
                        img2img_strength,
                        img2img_steps,
                        img2img_guidance,
                        img2img_seed
                    ],
                    outputs=[img2img_output,loading_status]
                )
                img2img_generate.interactive = loading_status.value == "Ready"

                load_to_input.click(
                fn=lambda x: x,
                inputs=img2img_output,
                outputs=img2img_input
                )
    return demo

demo = ui()
# demo.launch(debug=True)
demo.launch(share=True)

RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Could not import module 'CLIPImageProcessor'. Are this object's requirements defined correctly?

In [ ]:
demo.close()